In [ ]:
import tempfile

# connect to existing lakeshak or create if one doesn't exist
tmp_pth = tempfile.mkdtemp()

In [1]:
import sys; sys.path.append('/home/tyler/repos/binance-public-data/python/rsch')

from notebook_setup import *

setting up ipython for notebook
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


done configuring ipython

working directory : /home/tyler/repos/binance-public-data/python/rsch/notebooks

Python 3.10.13 | packaged by conda-forge | (main, Oct 26 2023, 18:07:37) [GCC 12.3.0]
Type 'copyright', 'credits' or 'license' for more information
IPython 8.15.0 -- An enhanced Interactive Python. Type '?' for help.



In [58]:

from lakeshack import LakeShack
import pandas as pd

tmp_pth = '/tmp/tmpsm7tqw4m'
lakeshack = LakeShack(tmp_pth)

data_people = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve']
})
lakeshack.write_deltalake('people', data_people)

data_purchases = pd.DataFrame({
    'purchase_id': [101, 102, 103, 104, 105],
    'buyer_id': [3, 1, 2, 5, 4],
    'amount': [250, 75, 120, 150, 200]
})
lakeshack.write_deltalake('purchases', data_purchases)

# Perform a join query between 'people' and 'purchases'
join_query = """
SELECT p.name, pu.amount
FROM people p
JOIN purchases pu ON p.id = pu.buyer_id
"""
join_result = lakeshack.sql(join_query)

TypeError: unhashable type: 'set'

In [4]:
join_result

,name,amount
0,Alice,75
1,Charlie,250
2,David,200
3,Eve,150
4,Alice,75
5,Charlie,250
6,David,200
7,Eve,150
8,Alice,75
9,Charlie,250


In [5]:
people_table = lakeshack.tables['people']

# Access table properties
table_name = people_table.table_uri
table_schema = people_table.schema()
table_version = people_table.version()
table_files = people_table.files()
table_history = people_table.history()

print(table_name)
print(table_schema)
print(table_version)
print(table_files)
print(table_history)


/tmp/tmpsm7tqw4m/people/
Schema([Field(id, PrimitiveType("long"), nullable=True), Field(name, PrimitiveType("string"), nullable=True)])
3
['part-00001-7ab11c7f-102e-4b11-9787-6f6d7720d61b-c000.snappy.parquet', '3-76652694-7575-4f8a-b03e-e334bc1eb99f-0.parquet']
[{'timestamp': 1703828039094, 'operation': 'WRITE', 'operationParameters': {'mode': 'Append', 'partitionBy': '[]'}, 'clientVersion': 'delta-rs.0.17.0', 'version': 3}, {'timestamp': 1703810637139, 'operation': 'DELETE', 'operationParameters': {'predicate': "name = 'Bob'"}, 'clientVersion': 'delta-rs.0.17.0', 'version': 2}, {'timestamp': 1703810079095, 'operation': 'WRITE', 'operationParameters': {'partitionBy': '[]', 'mode': 'Append'}, 'clientVersion': 'delta-rs.0.17.0', 'version': 1}, {'timestamp': 1703809309235, 'operation': 'CREATE TABLE', 'operationParameters': {'location': 'file:///tmp/tmpsm7tqw4m/people', 'protocol': '{"minReaderVersion":1,"minWriterVersion":2,"readerFeatures":null,"writerFeatures":null}', 'metadata': '{"co

In [6]:
people_table.file_uris()

['/tmp/tmpsm7tqw4m/people/part-00001-7ab11c7f-102e-4b11-9787-6f6d7720d61b-c000.snappy.parquet',
 '/tmp/tmpsm7tqw4m/people/3-76652694-7575-4f8a-b03e-e334bc1eb99f-0.parquet']

In [9]:
pd.DataFrame(people_table.history())

,timestamp,operation,operationParameters,clientVersion,version
0,1703828039094,WRITE,"{'mode': 'Append', 'partitionBy': '[]'}",delta-rs.0.17.0,3
1,1703810637139,DELETE,{'predicate': 'name = 'Bob''},delta-rs.0.17.0,2
2,1703810079095,WRITE,"{'partitionBy': '[]', 'mode': 'Append'}",delta-rs.0.17.0,1
3,1703809309235,CREATE TABLE,"{'metadata': '{""configuration"":{},""created_tim...",delta-rs.0.17.0,0


In [10]:
lakeshack.sql('SELECT * FROM people')

,id,name
0,1,Alice
1,3,Charlie
2,4,David
3,5,Eve
4,1,Alice
5,3,Charlie
6,4,David
7,5,Eve
8,1,Alice
9,2,Bob


In [13]:
people_table.load_version(2)

In [17]:
people_table.load_version??

Signature: people_table.load_version(version: int) -> None
Source:   
    def load_version(self, version: int) -> None:
        """
        Load a DeltaTable with a specified version.

        Args:
            version: the identifier of the version of the DeltaTable to load
        """
        self._table.load_version(version)
File:      ~/anaconda3/envs/pyks2/lib/python3.10/site-packages/deltalake/table.py
Type:      method

In [18]:
lakeshack.sql('SELECT * FROM people', use_latest_versions=False)

,id,name
0,1,Alice
1,3,Charlie
2,4,David
3,5,Eve
4,1,Alice
5,3,Charlie
6,4,David
7,5,Eve
8,1,Alice
9,2,Bob


In [25]:
people_table.delete("name = 'Bob'")
lakeshack.sql('select * from people')

,id,name
0,1,Alice
1,2,Bob
2,3,Charlie
3,4,David
4,5,Eve


In [19]:
people_table.to_pandas()

,id,name
0,1,Alice
1,3,Charlie
2,4,David
3,5,Eve
4,1,Alice
5,3,Charlie
6,4,David
7,5,Eve


In [23]:
ds = people_table.to_pyarrow_dataset()

import duckdb

duckdb.sql('select * from ds')

┌───────┬─────────┐
│  id   │  name   │
│ int64 │ varchar │
├───────┼─────────┤
│     1 │ Alice   │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
│     1 │ Alice   │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
└───────┴─────────┘

In [30]:
people_table.load_version(3)
ds = people_table.to_pyarrow_dataset()

duckdb.sql('select * from ds')


┌───────┬─────────┐
│  id   │  name   │
│ int64 │ varchar │
├───────┼─────────┤
│     1 │ Alice   │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
│     1 │ Alice   │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
│     1 │ Alice   │
│     2 │ Bob     │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
├───────┴─────────┤
│     13 rows     │
└─────────────────┘

In [35]:
lakeshack.conn.get_table_names('select * from people')

set()

In [49]:
lakeshack.conn.sql('select * from people')
lakeshack.conn.sql('show tables')

┌───────┬─────────┐
│  id   │  name   │
│ int64 │ varchar │
├───────┼─────────┤
│     1 │ Alice   │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
│     1 │ Alice   │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
│     1 │ Alice   │
│     2 │ Bob     │
│     3 │ Charlie │
│     4 │ David   │
│     5 │ Eve     │
├───────┴─────────┤
│     13 rows     │
└─────────────────┘

┌───────────┐
│   name    │
│  varchar  │
├───────────┤
│ people    │
│ purchases │
└───────────┘

In [52]:
duckdb.get_table_names('select * from people left join purchases on people.id = purchases.buyer_id')

{'people', 'purchases'}

In [54]:
lakeshack.conn.get_table_names('select * from people left join purchases on people.id = purchases.buyer_id')

set()

In [55]:
lakeshack.conn.table?

Docstring:
table(self: duckdb.duckdb.DuckDBPyConnection, table_name: str) -> duckdb.duckdb.DuckDBPyRelation

Create a relation object for the name'd table
Type:      method

In [56]:
duckdb.table?

Docstring:
table(table_name: str, connection: duckdb.DuckDBPyConnection = None) -> duckdb.duckdb.DuckDBPyRelation

Create a relation object for the name'd table
Type:      builtin_function_or_method

In [12]:
lakeshack.tables['people'].table_uri

'/tmp/tmpsm7tqw4m/people/'

In [3]:
str(pd.Timestamp.utcnow().isoformat())

'2023-12-29T00:32:06.052537+00:00'

In [11]:
pd.Timestamp.utcnow().isoformat(timespec='seconds')

# utcnow.isoformat(timespec='seconds')

'2023-12-29T00:34:05+00:00'

In [13]:
lakeshack.sql('show tables')

ValueError: Failed to parse datetime string: premature end of input

In [2]:
import tempfile


tmp_pth = tempfile.mkdtemp()

In [3]:
tmp_pth

'/tmp/tmpsm7tqw4m'